# Liberías

In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn import tree,svm,neighbors
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import OrdinalEncoder

import matplotlib.pyplot as plt

from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import RMSprop
from keras.constraints import maxnorm

In [3]:
import tensorflow as tf
# from tf.compat.v1.keras.backend.set_session import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

Device mapping: no known devices.


# Lectura de datos

In [4]:
data = pd.read_csv('C:/Users/Enrique/OneDrive - UFV/Escritorio/CEIEC/RASFF/splited_RASFF_DATA_16092019.csv', sep=';', 
                header=0, index_col = 0)
data ['DATE_CASE'] = data ['DATE_CASE'].astype(str)
data ['HAZARDS_CAT'] = data ['HAZARDS_CAT'].astype(str)
data = data.sample(frac = 1)
train  = data
train

,DATE_CASE,NOT_COUNTRY,PROD_CAT,TYPE,RISK_DECISION,ACTION_TAKEN,DISTRIBUTION_STAT,HAZARDS_CAT,COUNT_ORIGEN,COUNT_DESTIN,COUNT_CONCERN
43622,2016-07-15,United Kingdom,"nuts, nut products and seeds",food,serious,import not authorised,product not (yet) placed on the market,mycotoxins,China,,United Kingdom
54761,2015-06-12,Czech Republic,"dietetic foods, food supplements, fortified foods",food,serious,withdrawal from the market,distribution to other member countries,composition,India,Austria,INFOSAN
107835,2009-07-03,France,meat and meat products (other than poultry),food,undecided,withdrawal from the market,distribution on the market (possible),microbial contaminants (other),Brazil,Germany,Netherlands
18959,2018-05-23,France,"dietetic foods, food supplements, fortified foods",food,serious,recall from consumers,distribution to other member countries,pathogenic micro-organisms,Taiwan,Spain,INFOSAN
4807,2019-04-18,Belgium,food contact materials,fcm,serious,public warning - press release,distribution restricted to notifying country,industrial contaminants,Hong Kong,Belgium,INFOSAN
...,...,...,...,...,...,...,...,...,...,...,...
127197,2006-07-25,Greece,food contact materials,food,undecided,re-dispatch,no distribution,organoleptic aspects,Hong Kong,,Greece
5020,2019-04-16,Spain,"dietetic foods, food supplements, fortified foods",food,not serious,destruction,distribution to other member countries,novel food,Spain,Estonia,Germany
3224,2019-06-14,Czech Republic,poultry meat and poultry meat products,food,undecided,withdrawal from the market,distribution to other member countries,pathogenic micro-organisms,Poland,Greece,
24913,2018-01-11,Germany,"dietetic foods, food supplements, fortified foods",food,serious,recall from consumers,distribution to other member countries,natural toxins (other),Spain,Germany,INFOSAN


# Features

In [5]:
features = [0,1,6,8]
target = [2]
X = train.iloc[:,features]
y = train.iloc[:,target]

In [6]:
X 

,DATE_CASE,NOT_COUNTRY,DISTRIBUTION_STAT,COUNT_ORIGEN
43622,2016-07-15,United Kingdom,product not (yet) placed on the market,China
54761,2015-06-12,Czech Republic,distribution to other member countries,India
107835,2009-07-03,France,distribution on the market (possible),Brazil
18959,2018-05-23,France,distribution to other member countries,Taiwan
4807,2019-04-18,Belgium,distribution restricted to notifying country,Hong Kong
...,...,...,...,...
127197,2006-07-25,Greece,no distribution,Hong Kong
5020,2019-04-16,Spain,distribution to other member countries,Spain
3224,2019-06-14,Czech Republic,distribution to other member countries,Poland
24913,2018-01-11,Germany,distribution to other member countries,Spain


In [7]:
y

,PROD_CAT
43622,"nuts, nut products and seeds"
54761,"dietetic foods, food supplements, fortified foods"
107835,meat and meat products (other than poultry)
18959,"dietetic foods, food supplements, fortified foods"
4807,food contact materials
...,...
127197,food contact materials
5020,"dietetic foods, food supplements, fortified foods"
3224,poultry meat and poultry meat products
24913,"dietetic foods, food supplements, fortified foods"


# Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Para probar codificaciones (opcional)

In [9]:
encX = OneHotEncoder(handle_unknown='ignore')
#encX = OrdinalEncoder()
#encX = FeatureHasher(n_features=25,input_type='string')
#encX = MultiLabelBinarizer()
encX.fit(X.values)
X_one_hot = encX.transform(X.values)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_one_hot, y, test_size=0.2)

# Árboles de decisión

In [11]:
params ={'criterion' :['gini', 'entropy'], 
         'splitter' :['best', 'random'], 
         'max_features':['auto', 'sqrt', 'log2']
        }
grid_search_cv = GridSearchCV(tree.DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)
clf = grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


In [12]:
clf.best_estimator_

DecisionTreeClassifier(max_features='auto', random_state=42, splitter='random')

In [13]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_features='auto',
                       random_state=42)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred, zero_division=0))

                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.89      0.86      0.88        86
                  animal by-products - (obsolete)       0.80      0.57      0.67         7
                    animal nutrition - (obsolete)       0.91      0.81      0.86       128
            bivalve molluscs and products thereof       0.83      0.76      0.79       492
                 cephalopods and products thereof       0.64      0.66      0.65       159
                      cereals and bakery products       0.84      0.80      0.82      1348
     cocoa and cocoa preparations, coffee and tea       0.81      0.88      0.84       615
                                   compound feeds       0.86      0.88      0.87       186
                                    confectionery       0.85      0.83      0.84       576
                 crustaceans and products thereof       0.72      0.73      0.73       51

In [15]:
clf.score(X_test, y_test)

0.8505970465625663

# Random Forest

In [20]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

rf = RandomForestClassifier()

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 288 candidates, totalling 864 fits


C:\Users\Enrique\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 110,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 200}

In [25]:
clf = RandomForestClassifier(bootstrap = True, max_depth = 110, max_features = 3, min_samples_leaf =  3, min_samples_split = 8, n_estimators =  200)
clf = clf.fit(X_train,y_train)

<ipython-input-25-50ed6adac834>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = clf.fit(X_train,y_train)


In [26]:
y_pred = clf.predict(X_test)

In [27]:
y_pred

array(['fruits and vegetables', 'fruits and vegetables',
       'fruits and vegetables', ...,
       'dietetic foods, food supplements, fortified foods',
       'dietetic foods, food supplements, fortified foods',
       'dietetic foods, food supplements, fortified foods'], dtype=object)

In [28]:
print(classification_report(y_test,y_pred))

C:\Users\Enrique\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                   precision    recall  f1-score   support

                              alcoholic beverages       0.00      0.00      0.00        86
                  animal by-products - (obsolete)       0.00      0.00      0.00         7
                    animal nutrition - (obsolete)       0.00      0.00      0.00       128
            bivalve molluscs and products thereof       0.00      0.00      0.00       492
                 cephalopods and products thereof       0.00      0.00      0.00       159
                      cereals and bakery products       1.00      0.02      0.04      1348
     cocoa and cocoa preparations, coffee and tea       0.00      0.00      0.00       615
                                   compound feeds       0.00      0.00      0.00       186
                                    confectionery       0.00      0.00      0.00       576
                 crustaceans and products thereof       0.00      0.00      0.00       51

C:\Users\Enrique\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Enrique\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
clf.score(X_test, y_test)

0.32816364021762173

# Gaussian naive Bayes clasiffier

In [ ]:
params = {'var_smoothing': np.logspace(0,-9, num=1)}
gs_NB =  GridSearchCV(GaussianNB(), params, verbose=1, cv=3)
gs_NB.fit(X_train.toarray(), y_train)

gs_NB.best_params_

In [ ]:
clf = GaussianNB(var_smoothing = 1.0)
clf = clf.fit(X_train_one_hot.toarray(),y_train)

In [ ]:
y_pred = clf.predict(X_test_one_hot.toarray())

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot.toarray(), y_test)

# Regresión logística

In [ ]:
grid={"C":np.logspace(-3,3,7), "penalty":["l2"]}
clf = GridSearchCV(LogisticRegression(), grid, cv=10)
clf.fit(X_train, np.ravel(y_train))

C:\Users\Enrique\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Enrique\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_i

In [ ]:
clf.best_params

In [ ]:
clf = LogisticRegression()
clf = clf.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test, y_test)

# Support vector machine

In [ ]:
clf = svm.SVC()
clf = clf.fit(X_train_one_hot,y_train)
y_pred = clf.predict(X_test_one_hot)
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot, y_test)

# Boosted Trees

In [ ]:
clf = GradientBoostingClassifier()
clf = clf.fit(X_train_one_hot,y_train)
y_pred = clf.predict(X_test_one_hot)
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot, y_test)

In [ ]:
clf = AdaBoostClassifier()
clf = clf.fit(X_train_one_hot,y_train)
y_pred = clf.predict(X_test_one_hot)
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot, y_test)

# Nearest centroid

In [ ]:
clf = NearestCentroid()
clf = clf.fit(X_train_one_hot,y_train)
y_pred = clf.predict(X_test_one_hot)
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot, y_test)

# Nearest neighbor

In [ ]:
clf = neighbors.KNeighborsClassifier()
clf = clf.fit(X_train_one_hot,y_train)
y_pred = clf.predict(X_test_one_hot)
print(classification_report(y_test,y_pred))

In [ ]:
clf.score(X_test_one_hot, y_test)

# MLP sin embeddings

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y_train.values)
Y_train_label = le.transform(y_train.values) 
y_train_one_hot = to_categorical(Y_train_label)
display(y_train_one_hot)

In [ ]:
y_test_label = le.transform(y_test.values) 
y_test_one_hot = to_categorical(y_test_label)
display(y_test_one_hot)

In [ ]:
model = Sequential() 
model.add(Dense(300, activation='relu', input_dim=351))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(38)) 
optimizer = RMSprop(lr=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics= ['accuracy'])

In [ ]:
hist = model.fit(X_train_one_hot,y_train_one_hot, epochs=70,validation_data=(X_test_one_hot, y_test_one_hot), batch_size=120)

In [ ]:
plt.plot(hist.history['loss']) 
plt.plot(hist.history['val_loss']) 
plt.show()


plt.plot(hist.history['acc']) 
plt.plot(hist.history['val_acc']) 
plt.show()

## Pruebas grid search

In [ ]:
def create_model(neuronas1 = 25,neuronas2 =25,neuronas3 = 25):
           
    model = Sequential()
    activation='relu'
    init_mode='normal'
    dropout_rate=0.0
    weight_constraint=0
    model.add(Dense(neuronas1, activation=activation, input_dim=374,kernel_initializer=init_mode))
    model.add(Dense(neuronas2, activation=activation,kernel_initializer=init_mode))
    model.add(Dense(neuronas3, activation=activation,kernel_initializer=init_mode))
    model.add(Dropout(dropout_rate))
    model.add(Dense(38,kernel_initializer=init_mode))
    optimizer = RMSprop(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model

seed = 7
np.random.seed(seed)
model = KerasClassifier(build_fn=create_model, verbose=1,batch_size=120, epochs = 70)

In [ ]:
neuronas1 = [25,75,100,150,200,250,300,350,400]
neuronas2 = [25,75,100,150,200,250,300,350,400]
neuronas3 = [25,75,100,150,200,250,300,350,400]
param_grid = dict(neuronas1=neuronas1,neuronas2=neuronas2,neuronas3=neuronas3)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=2)
grid_result = grid.fit(X_train_one_hot, y_train_one_hot)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))